In [195]:
"""TPAW Planner Export Using Existing Simulator

This notebook runs a single-trial simulation using the existing engine and
post-processes the results into real (inflation-adjusted) post-tax income
streams suitable for TPAW Planner.
"""

from pathlib import Path
from dataclasses import dataclass
import pandas as pd

from app.models.simulator import SimulationEngine
from app.data.constants import INTERVALS_PER_YEAR

In [196]:
# Run a single simulation trial and get the first trial DataFrame
engine = SimulationEngine(trial_qty=1)
engine.gen_all_trials()

results = engine.results
trial_dfs = results.as_dataframes()

assert len(trial_dfs) == 1, "Expected exactly one trial DataFrame"
df = trial_dfs[0]

print("DataFrame columns:")
print(df.columns.tolist())
print(f"Number of intervals: {len(df)}")
print(df.head())


DataFrame columns:
['Date', 'Net Worth', 'Inflation', 'Job Income', 'SS User', 'SS Partner', 'Pension', 'Total Income', 'Spending', 'Kids', 'Income Taxes', 'Medicare Taxes', 'Social Security Taxes', 'Portfolio Taxes', 'Total Taxes', 'Total Costs', 'Portfolio Return', 'Annuity', 'Net Transaction', 'US_Stock_%', 'US_Bond_%', 'Intl_ex_US_Stock_%', 'TIPS_%', '10_yr_Treasury_%', 'Long_term_Treasury_%', 'Short_term_Treasury_%', 'Mid_term_Treasury_%', 'REIT_%', 'Commodities_%', 'Gold_%', 'US_Stock_rate', 'US_Bond_rate', 'Intl_ex_US_Stock_rate', 'TIPS_rate', '10_yr_Treasury_rate', 'Long_term_Treasury_rate', 'Short_term_Treasury_rate', 'Mid_term_Treasury_rate', 'REIT_rate', 'Commodities_rate', 'Gold_rate']
Number of intervals: 217
      Date    Net Worth  Inflation  Job Income  SS User  SS Partner  Pension  \
0  2025.75  1764.000000   1.000000       0.000      0.0         0.0      0.0   
1  2026.00  1750.007618   1.001673      27.375      0.0         0.0      0.0   
2  2026.25  1651.184375   1.

/Users/chris/Projects/LifeFInances/app/models/controllers/economic_data.py:172: RuntimeWarning: covariance is not symmetric positive-semidefinite.
  yield_matrix = np.random.multivariate_normal(


In [197]:
# Convert nominal quarterly flows to real yearly flows per stream

job_q = df["Job Income"]
ss_user_q = df["SS User"]
ss_partner_q = df["SS Partner"]
pension_q = df["Pension"]

infl = df["Inflation"]

# Real quarterly = nominal quarterly / inflation
job_real_q = job_q / infl
ss_user_real_q = ss_user_q / infl
ss_partner_real_q = ss_partner_q / infl
pension_real_q = pension_q / infl

# Real yearly
job_real_y = job_real_q * INTERVALS_PER_YEAR
ss_user_real_y = ss_user_real_q * INTERVALS_PER_YEAR
ss_partner_real_y = ss_partner_real_q * INTERVALS_PER_YEAR
pension_real_y = pension_real_q * INTERVALS_PER_YEAR

streams_real_y = pd.DataFrame({
    "Date": df["Date"],
    "job_real_y": job_real_y,
    "ss_user_real_y": ss_user_real_y,
    "ss_partner_real_y": ss_partner_real_y,
    "pension_real_y": pension_real_y,
})

streams_real_y


,Date,job_real_y,ss_user_real_y,ss_partner_real_y,pension_real_y
0,2025.75,0.000000,0.000000,0.000000,0.0
1,2026.00,109.317135,0.000000,0.000000,0.0
2,2026.25,108.708184,0.000000,0.000000,0.0
3,2026.50,187.478192,0.000000,0.000000,0.0
4,2026.75,186.415070,0.000000,0.000000,0.0
...,...,...,...,...,...
212,2078.75,0.000000,34.810234,24.972509,65.7
213,2079.00,0.000000,34.810234,24.972509,65.7
214,2079.25,0.000000,34.810234,24.972509,65.7
215,2079.50,0.000000,34.810234,24.972509,65.7


In [198]:
# Compute total real yearly taxes and apportion across streams

income_tax_q = df["Income Taxes"]
medicare_tax_q = df["Medicare Taxes"]
ss_tax_q = df["Social Security Taxes"]
portfolio_tax_q = df["Portfolio Taxes"]

# Total nominal tax per interval
total_tax_q = income_tax_q + medicare_tax_q + ss_tax_q + portfolio_tax_q

# Convert to real yearly taxes
total_tax_real_q = total_tax_q / infl
total_tax_real_y = total_tax_real_q * INTERVALS_PER_YEAR

# Total pre-tax real yearly income across streams
total_income_real_y = (
    job_real_y
    + ss_user_real_y
    + ss_partner_real_y
    + pension_real_y
)

# Avoid division by zero when total income is 0
eps = 1e-9

share_job = job_real_y / (total_income_real_y + eps)
share_ss_user = ss_user_real_y / (total_income_real_y + eps)
share_ss_partner = ss_partner_real_y / (total_income_real_y + eps)
share_pension = pension_real_y / (total_income_real_y + eps)

# When total_income_real_y is effectively zero, set all shares to 0
zero_mask = total_income_real_y.abs() < eps
share_job[zero_mask] = 0.0
share_ss_user[zero_mask] = 0.0
share_ss_partner[zero_mask] = 0.0
share_pension[zero_mask] = 0.0

job_tax_real_y = share_job * total_tax_real_y
ss_user_tax_real_y = share_ss_user * total_tax_real_y
ss_partner_tax_real_y = share_ss_partner * total_tax_real_y
pension_tax_real_y = share_pension * total_tax_real_y

apportioned = pd.DataFrame({
    "Date": df["Date"],
    "total_tax_real_y": total_tax_real_y,
    "job_tax_real_y": job_tax_real_y,
    "ss_user_tax_real_y": ss_user_tax_real_y,
    "ss_partner_tax_real_y": ss_partner_tax_real_y,
    "pension_tax_real_y": pension_tax_real_y,
})

apportioned


,Date,total_tax_real_y,job_tax_real_y,ss_user_tax_real_y,ss_partner_tax_real_y,pension_tax_real_y
0,2025.75,0.000000,0.000000,0.000000,0.000000,0.000000
1,2026.00,-13.683865,-13.683865,-0.000000,-0.000000,-0.000000
2,2026.25,-13.550663,-13.550663,-0.000000,-0.000000,-0.000000
3,2026.50,-37.486369,-37.486369,-0.000000,-0.000000,-0.000000
4,2026.75,-37.024189,-37.024189,-0.000000,-0.000000,-0.000000
...,...,...,...,...,...,...
212,2078.75,-2.493643,-0.000000,-0.691763,-0.496264,-1.305617
213,2079.00,-2.447068,-0.000000,-0.678842,-0.486995,-1.281231
214,2079.25,-2.428730,-0.000000,-0.673755,-0.483345,-1.271629
215,2079.50,-2.386890,-0.000000,-0.662148,-0.475019,-1.249723


In [199]:
# Derive real yearly post-tax streams per source and validate totals

job_post_real_y = job_real_y + job_tax_real_y
ss_user_post_real_y = ss_user_real_y + ss_user_tax_real_y
ss_partner_post_real_y = ss_partner_real_y + ss_partner_tax_real_y
pension_post_real_y = pension_real_y + pension_tax_real_y

total_post_real_y_streams = (
    job_post_real_y
    + ss_user_post_real_y
    + ss_partner_post_real_y
    + pension_post_real_y
)

# Consistency check: total post-tax income vs income + taxes
total_income_plus_tax_real_y = total_income_real_y + total_tax_real_y

check_df = pd.DataFrame({
    "Date": df["Date"],
    "total_income_real_y": total_income_real_y,
    "total_tax_real_y": total_tax_real_y,
    "total_post_real_y_streams": total_post_real_y_streams,
    "total_income_plus_tax_real_y": total_income_plus_tax_real_y,
})

diff = (check_df["total_post_real_y_streams"] - check_df["total_income_plus_tax_real_y"]).abs().max()
assert diff < 1.0, f"Max diff is too high: {diff}"
print("Max abs diff between streams-sum and income+tax totals: ", diff)

post_tax_streams = pd.DataFrame({
    "Date": df["Date"],
    "job_post_real_y": job_post_real_y,
    "ss_user_post_real_y": ss_user_post_real_y,
    "ss_partner_post_real_y": ss_partner_post_real_y,
    "pension_post_real_y": pension_post_real_y,
})

post_tax_streams


Max abs diff between streams-sum and income+tax totals:  2.0045831661263946e-10


,Date,job_post_real_y,ss_user_post_real_y,ss_partner_post_real_y,pension_post_real_y
0,2025.75,0.000000,0.000000,0.000000,0.000000
1,2026.00,95.633270,0.000000,0.000000,0.000000
2,2026.25,95.157521,0.000000,0.000000,0.000000
3,2026.50,149.991823,0.000000,0.000000,0.000000
4,2026.75,149.390881,0.000000,0.000000,0.000000
...,...,...,...,...,...
212,2078.75,0.000000,34.118471,24.476245,64.394383
213,2079.00,0.000000,34.131391,24.485514,64.418769
214,2079.25,0.000000,34.136478,24.489163,64.428371
215,2079.50,0.000000,34.148085,24.497490,64.450277


In [200]:
# Group consecutive intervals into periods per stream

GROUP_TOL = 3.0 # in thousands, higher values mean less periods

@dataclass
class Period:
    start_date_dec: float
    end_date_dec: float
    amount_real_y: float


def group_stream(date_series, value_series):
    periods: list[Period] = []
    if len(value_series) == 0:
        return periods

    current_value = value_series.iloc[0]
    start_idx = 0

    for i in range(1, len(value_series)):
        if abs(value_series.iloc[i] - current_value) > GROUP_TOL:
            periods.append(
                Period(
                    start_date_dec=date_series.iloc[start_idx],
                    end_date_dec=date_series.iloc[i - 1],
                    amount_real_y=current_value,
                )
            )
            current_value = value_series.iloc[i]
            start_idx = i

    periods.append(
        Period(
            start_date_dec=date_series.iloc[start_idx],
            end_date_dec=date_series.iloc[len(value_series) - 1],
            amount_real_y=current_value,
        )
    )

    # Drop periods that are effectively zero
    nonzero_periods = [p for p in periods if abs(p.amount_real_y) > 1.0]
    return nonzero_periods


job_periods = group_stream(df["Date"], job_post_real_y)
ss_user_periods = group_stream(df["Date"], ss_user_post_real_y)
ss_partner_periods = group_stream(df["Date"], ss_partner_post_real_y)
pension_periods = group_stream(df["Date"], pension_post_real_y)

len(job_periods), len(ss_user_periods), len(ss_partner_periods), len(pension_periods)

(17, 1, 1, 1)

In [201]:
# Convert decimal dates to "Month Year" and build TPAW-ready tables

MONTH_NAMES = [
    "Jan",
    "Feb",
    "Mar",
    "Apr",
    "May",
    "Jun",
    "Jul",
    "Aug",
    "Sep",
    "Oct",
    "Nov",
    "Dec",
]


def decimal_date_to_month_year(decimal_date, is_end_date=False):
    """Convert decimal date (e.g., 2025.5) to 'Month Year' format (e.g., 'July 2025').

    Uses quarter mapping consistent with simulator intervals:
    - Start: Jan/Apr/Jul/Oct
    - End:   Mar/Jun/Sep/Dec
    """
    year = int(decimal_date)
    quarter_decimal = decimal_date % 1
    if is_end_date:
        month_num = int(quarter_decimal * 12) + 3
    else:
        month_num = int(quarter_decimal * 12) + 1
    month_name = MONTH_NAMES[month_num - 1]
    return f"{month_name} {year}"


def periods_to_df(periods, label):
    rows = []
    for p in periods:
        rows.append(
            {
                "Start Date": decimal_date_to_month_year(p.start_date_dec, is_end_date=False),
                "End Date": decimal_date_to_month_year(p.end_date_dec, is_end_date=True),
                # Round to nearest dollar, full dollars for TPAW
                "Post-Tax Real Income (per year)": int(round(p.amount_real_y*1000)),
                "Stream": label,
            }
        )
    return pd.DataFrame(rows)


job_df = periods_to_df(job_periods, "Job")
ss_user_df = periods_to_df(ss_user_periods, "SS User")
ss_partner_df = periods_to_df(ss_partner_periods, "SS Partner")
pension_df = periods_to_df(pension_periods, "Pension") if pension_periods else pd.DataFrame()

print("Job income periods:")
print(job_df.to_string(index=False))

print("\nSS User periods:")
print(ss_user_df.to_string(index=False))

print("\nSS Partner periods:")
print(ss_partner_df.to_string(index=False))

if not pension_df.empty:
    print("\nPension periods:")
    print(pension_df.to_string(index=False))


Job income periods:
Start Date End Date  Post-Tax Real Income (per year) Stream
  Jan 2026 Jun 2026                            95633    Job
  Jul 2026 Dec 2026                           149992    Job
  Jan 2027 Dec 2028                           153363    Job
  Jan 2029 Jun 2029                           157105    Job
  Jul 2029 Dec 2030                           152459    Job
  Jan 2032 Jun 2032                            79428    Job
  Jul 2032 Dec 2032                           129922    Job
  Jan 2033 Dec 2033                           133326    Job
  Jan 2034 Dec 2035                           139385    Job
  Jan 2036 Sep 2036                           143172    Job
  Oct 2036 Dec 2036                           139890    Job
  Jan 2038 Jun 2038                            70820    Job
  Jul 2038 Dec 2038                           115152    Job
  Jan 2039 Jun 2039                           118569    Job
  Jul 2039 Sep 2040                           114519    Job
  Oct 2040 Dec 2042 